### Start and Run All

In [2]:
import os
import pandas as pd
from urllib import request
from bs4 import BeautifulSoup
# from datetime import date, timedelta
from datetime import datetime
from time import sleep, strftime
current_time = datetime.now()
current_time

datetime.datetime(2025, 1, 20, 20, 29, 47, 167331)

In [3]:
url = "https://www.set.or.th/en/market/product/stock/quote/"
percent_span = '' 
input_file = 'name-ttl.csv'
current_dir = '../Daily'
input_dir = os.path.join(os.path.dirname(current_dir), 'Data')
output_dir = os.path.join(os.path.dirname(current_dir), 'Data')
print(input_dir, output_dir)

..\Data ..\Data


In [4]:
file_in = os.path.join(input_dir, input_file)
file_out = os.path.join(output_dir, "price-hilo.csv")
print(file_in, file_out)

..\Data\name-ttl.csv ..\Data\price-hilo.csv


In [5]:
def cvt_str_to_flt(input_str):
    # Check for the special pattern and return 0
    if input_str == '(-)':
        return 0
    
    # Remove parentheses and percent sign
    cleaned_str = input_str.replace('(', '').replace(')', '').replace('%', '')

    # Convert to float
    try:
        return float(cleaned_str)
    except ValueError:
        # Handle the case where the conversion fails
        return None
 

In [6]:
def get_stock_data(in_df):

    in_df.columns = ['name']
    
    output_columns = ['name', 'price', 'today_low', 'today_high', 'year_low', 'year_high', 'percent', 'updated_at']
    # Initialize an empty DataFrame with specified columns
    out_df = pd.DataFrame(columns=output_columns)
    
    for index, row in in_df.iterrows():
        name = str(row['name']).upper()
        response = request.urlopen(url + name + '/price')
        html_page = BeautifulSoup(response, 'html.parser')
        
  #      price = html_page.find('div', class_='value text-white mb-0 me-2 lh-1 stock-info').text.strip()        
  #      if (price == '-'):
  #          price = '0.00'        

        price_div = html_page.find('div', class_='value text-white mb-0 me-2 lh-1 stock-info')
        if price_div:
            price = price_div.text.strip()
        else:
            print(f"Price div not found for {name}")
            price = '0.00'     
            
        # Try to find the <h3> tag with the class 'theme-success'
        h3_tag = html_page.find('h3', class_='theme-success')
        if h3_tag:
        # Find all <span> tags within the found <h3> tag
            span_tags = h3_tag.find_all('span')
    
            if len(span_tags) > 1:
                # Extract the text from the second <span> tag
                percent_span = span_tags[1].text
                # print(percent_span)
                
        # Try to find the <h3> tag with the class 'theme-danger'
        h3_tag = html_page.find('h3', class_='theme-danger')
        if h3_tag:
        # Find all <span> tags within the found <h3> tag
            span_tags = h3_tag.find_all('span')
    
            if len(span_tags) > 1:
                # Extract the text from the second <span> tag
                percent_span = span_tags[1].text
                # print(percent_span)    
                
        # Try to find the <h3> tag with the class 'theme-danger'
        h3_tag = html_page.find('h3', class_='theme-normal')
        if h3_tag:
        # Find all <span> tags within the found <h3> tag
            span_tags = h3_tag.find_all('span')
    
            if len(span_tags) > 1:
                # Extract the text from the second <span> tag
                percent_span = span_tags[1].text
                # print(percent_span)   
                
        percent = cvt_str_to_flt(percent_span)
                
        values_low = html_page.find_all('span', class_='title-font-family fs-16px fw-bolder me-auto lh-1')
        today_low = values_low[0].text
        if (today_low == '-'):
            today_low = '0.00'
        
        values_high = html_page.find_all('span', class_='title-font-family fs-16px fw-bolder lh-1')
        today_high = values_high[0].text
        if (today_high == '-'):
            today_high = '0.00'         
        
        year_low = values_low[1].text        
        year_high = values_high[1].text     
        
        # Find the div with the specific class, then find the span within it
        div = html_page.find('div', class_='price-detail-date d-flex align-items-center text-nowrap fs-12px')
        if div:
            span = div.find('span')
            if span:
                text = span.get_text()
#                 print(text)
            else:
                print("Span not found")
        else:
            print("Div not found")
            
        # Extract the date part from the string
        # Assuming the date is always in the format 'dd Mon yyyy'
        date_str = ' '.join(text.split()[2:5])

        # Parse the date string into a datetime object
        date_obj = datetime.strptime(date_str, '%d %b %Y')

        # Format the datetime object to 'yyyy-mm-dd'
        updated_at = date_obj.strftime('%Y-%m-%d')

#         print(updated_at)
        
        # Append row to the DataFrame and print the row
        out_df.loc[len(out_df)] = [name, price, today_low, today_high, year_low, year_high, percent, updated_at]
#       print(out_df.iloc[-1].to_string(header=False, index=False)) 
        print(','.join(map(str, out_df.iloc[-1])))
        sleep(1)
        
    return out_df  # return the final dataframe with all values populated

In [7]:
start_time = strftime("%I:%M %p")
print('Start at: ', start_time)

Start at:  08:29 PM


In [8]:
df = pd.read_csv(file_in, header=None)
out_df = get_stock_data(df)

ACE,1.27,1.26,1.28,1.25,1.60,0.0,2025-01-20
ADVANC,283.00,282.00,286.00,193.00,299.00,-0.35,2025-01-20
AEONTS,114.50,114.50,116.50,107.00,168.50,-0.43,2025-01-20
AH,14.90,14.90,15.20,14.30,30.50,-0.67,2025-01-20
AIE,0.75,0.74,0.80,0.77,1.51,-3.85,2025-01-20
AIMIRT,10.50,10.50,10.60,10.20,11.20,0.0,2025-01-20
AIT,4.08,3.98,4.08,3.62,5.05,2.0,2025-01-20
AJ,2.40,2.38,2.44,2.44,7.15,-2.44,2025-01-20
AMATA,26.25,25.50,26.75,20.10,31.00,1.94,2025-01-20
ANAN,0.50,0.48,0.51,0.50,1.03,-1.96,2025-01-20
AOT,57.50,56.75,57.50,55.00,67.50,0.44,2025-01-20
AP,7.70,7.65,7.75,7.50,11.40,0.65,2025-01-20
ASIAN,7.65,7.65,7.70,6.35,11.20,-0.65,2025-01-20
ASK,8.40,8.40,8.50,8.40,20.30,-0.59,2025-01-20
ASP,2.30,2.30,2.32,2.12,2.88,-0.86,2025-01-20
ASW,7.80,7.70,7.80,7.35,8.45,0.0,2025-01-20
AWC,3.04,3.02,3.16,2.88,4.58,-3.18,2025-01-20
BA,19.00,18.90,19.40,14.20,26.00,-1.55,2025-01-20
BAM,5.50,5.40,5.60,5.35,10.40,0.92,2025-01-20
BANPU,5.10,5.05,5.15,4.38,7.50,0.0,2025-01-20
BAY,24.10,24.10,24.30,22.20,29.00

In [9]:
end_time = strftime("%I:%M %p")
print('End at: ', end_time)

End at:  08:35 PM


In [10]:
out_df.query('price == "-"')

,name,price,today_low,today_high,year_low,year_high,percent,updated_at
25,BCT,-,0.00,0.00,53.00,76.75,0.0,2025-01-20
103,KYE,-,0.00,0.00,280.00,342.00,0.0,2025-01-20
210,TYCN,-,0.00,0.00,1.84,2.78,0.0,2025-01-20


In [11]:
out_df.shape

(221, 8)

In [12]:
out_df = out_df[out_df['name'] != 'BCT']

In [13]:
out_df = out_df[out_df['name'] != 'LPF']

In [14]:
out_df = out_df[out_df['name'] != 'TYCN']

In [15]:
out_df = out_df[out_df['name'] != 'GRAMMY']

In [16]:
out_df = out_df[out_df['name'] != 'KYE']

In [17]:
out_df.shape

(217, 8)

In [18]:
out_df['price'] = out_df['price'].astype('float')
out_df['today_low'] = out_df['today_low'].astype('float')
out_df['today_high'] = out_df['today_high'].astype('float')
out_df['year_low'] = out_df['year_low'].astype('float')
out_df['year_high'] = out_df['year_high'].astype('float')
out_df['percent'] = out_df['percent'].astype('float')

In [19]:
sorted_out_df = out_df.sort_values('percent')

In [20]:
sorted_out_df.query('today_low < year_low')

,name,price,today_low,today_high,year_low,year_high,percent,updated_at
109,LIT,0.55,0.55,0.62,0.61,1.55,-12.70,2025-01-20
42,CENTEL,28.25,28.00,30.00,29.25,46.50,-5.83,2025-01-20
112,M,19.10,18.90,20.10,19.50,39.50,-4.50,2025-01-20
171,SPRC,5.70,5.65,5.95,5.75,9.40,-4.20,2025-01-20
4,AIE,0.75,0.74,0.80,0.77,1.51,-3.85,2025-01-20
111,LPN,2.12,2.12,2.18,2.16,3.92,-3.64,2025-01-20
38,BPP,8.60,8.60,8.90,8.85,15.20,-3.37,2025-01-20
43,CHG,2.16,2.12,2.26,2.20,3.24,-2.70,2025-01-20
7,AJ,2.40,2.38,2.44,2.44,7.15,-2.44,2025-01-20
73,GPSC,30.50,30.00,31.50,30.75,56.50,-2.40,2025-01-20


In [21]:
out_df.query('today_low < year_low').shape[0]

24

In [22]:
sorted_out_df.query('today_high > year_high')

,name,price,today_low,today_high,year_low,year_high,percent,updated_at


In [23]:
out_df.query('today_high > year_high').shape[0]

0

In [24]:
file_in = os.path.join(input_dir, input_file)
file_out = os.path.join(output_dir, "price-hilo.csv")
print(file_in, file_out)

..\Data\name-ttl.csv ..\Data\price-hilo.csv


In [53]:
out_df.to_csv(file_out, index=False)  # write the output dataframe to a new CSV file

In [55]:
current_time

datetime.datetime(2025, 1, 20, 20, 29, 47, 167331)